In [1]:
import numpy as np
from vispy import app
from vispy import gloo
from vispy.io import imsave
import OpenGL.GL as gl
from scipy import misc

ImportError: No module named numpy

In [2]:
cloudImage = misc.imread('/Users/rachel/Downloads/cloud_frac_padded_623_812_70_4096_4096.png')
divisor = np.full((4096, 4096, 3), 255.0)
cloudScaledImage = np.divide(cloudImage, divisor).astype(np.float32)
cloudTex = gloo.Texture2D(cloudScaledImage)

In [4]:
c = app.Canvas(show=False, size=(4096,4096))

INFO: Could not import backend "PyQt4":
No module named PyQt4
INFO:vispy:Could not import backend "PyQt4":
No module named PyQt4
INFO: Could not import backend "PyQt5":
No module named PyQt5
INFO:vispy:Could not import backend "PyQt5":
No module named PyQt5
INFO: Could not import backend "PySide":
dlopen(/Library/Python/2.7/site-packages/PySide/QtOpenGL.so, 2): Library not loaded: libpyside-python2.7.1.2.dylib
  Referenced from: /Library/Python/2.7/site-packages/PySide/QtOpenGL.so
  Reason: image not found
INFO:vispy:Could not import backend "PySide":
dlopen(/Library/Python/2.7/site-packages/PySide/QtOpenGL.so, 2): Library not loaded: libpyside-python2.7.1.2.dylib
  Referenced from: /Library/Python/2.7/site-packages/PySide/QtOpenGL.so
  Reason: image not found


In [5]:
vertex = """
    attribute vec2 a_position;
	attribute vec2 a_texCoord;
	uniform vec2 u_resolution;
	varying vec2 v_texCoord;
	void main() {
		// convert the rectangle from pixels to 0.0 to 1.0
		vec2 zeroToOne = a_position / u_resolution;
		// convert from 0->1 to 0->2
		vec2 zeroToTwo = zeroToOne * 2.0;
		// convert from 0->2 to -1->+1 (clipspace)
		vec2 clipSpace = zeroToTwo - 1.0;
		gl_Position = vec4(clipSpace * vec2(1, -1), 0, 1);
        //gl_Position = vec4(zeroToOne * vec2(1, -1), 0, 1);
		// pass the texCoord to the fragment shader
		// The GPU will interpolate this value between points.
		v_texCoord = zeroToOne;
	} """

In [6]:
fragment = """
    precision mediump float;
	// our texture
	uniform sampler2D dataTexture;
	// the texCoords passed in from the vertex shader.
	varying vec2 v_texCoord;
	const vec3 lightPosition = vec3(20.0, 0.0, 0.0);
	const vec3 dataShape = vec3(623.0, 812.0, 70.0);
	const int nSlices = 30;
	const int nSlicesPerRow = 6;
	const float sliceW = 623.0;
	const float sliceH = 812.0;
	const float alphaCorrection = 2.0/81.0;
	const vec2 textureShape = vec2(4096.0, 4096.0);
	vec3 getDatumColor(float datum) {
        vec3 color = vec3(1, 1, 0);
        return color;
    }
    float getDatumAlpha(float datum) {
        return datum * alphaCorrection;
    }
    vec3 mapTo2D(in vec3 p, float sliceW, float sliceH, int nSlicesPerRow, int nSlices){
        float zLevel = floor(p.z * 90.0);
        float zIndex = floor(zLevel / float(nSlices));
        float sliceIndex = (zLevel - (zIndex * float(nSlices)));
        float oSliceRow = floor(sliceIndex / float(nSlicesPerRow));
        //Rows are now in the opposite order
        float sliceRow = 4.0 - oSliceRow;
        float sliceCol = sliceIndex - float(nSlicesPerRow) * oSliceRow;
        
        vec2 pxy = (p.xy + vec2(sliceCol, sliceRow)) * vec2(sliceW, sliceH);
        return vec3(pxy, zIndex);
    }
    float sample3DTexture(sampler2D slices, vec3 p, float sliceW, float sliceH, int nSlicesPerRow, int nSlices, float bias){
        if(any(greaterThan(p, vec3(1.0))) || any(lessThan(p, vec3(0.0))))
           return 0.0;
        vec3 position = mapTo2D(p, sliceW, sliceH, nSlicesPerRow, nSlices);
        float zTile = position.z;
        vec4 datumRGB = texture2D(slices, position.xy / vec2(4096.0), bias);
        float datum;
        if (zTile == 0.0){
            datum = datumRGB.r;
        }else if (zTile == 1.0){
            datum = datumRGB.g;
        }else if (zTile == 2.0){
            datum = datumRGB.b;
        }
        return datum;
    }
    float sample3DTexture(sampler2D slices, vec3 p, float sliceW, float sliceH, int nSlicesPerRow, int nSlices){
        return sample3DTexture(slices, p, sliceW, sliceH, nSlicesPerRow, nSlices, 0.0);
    }
    vec4 getRGBAfromDataTex(sampler2D tex, vec3 pos, vec3 dataShape, vec2 texShape){
        // pos.xyz = clamp(pos.xyz, 0.01,0.99);
        //float datum = sampleAs3DTexture(tex, pos, dataShape, texShape);
        float datum = sample3DTexture(tex, pos, sliceW, sliceH, nSlicesPerRow, nSlices);
        vec3 color = getDatumColor(datum);
        float alpha = getDatumAlpha(datum);
        return vec4(color.xyz, alpha);
    }
	float getPathRGBA(vec3 startPos, vec3 endPos, float steps, sampler2D tex){
        /* Calculates the total RGBA values of a given path through a texture */
        //The direction from the front position to back position.
        vec3 dir = endPos - startPos;
        //vec3 dir = vec3(0.0,0.0,1.0);
    
        float rayLength = length(dir);
    
        //Calculate how long to increment in each step.
        float delta = 1.0 / steps;
        //The increment in each direction for each step.
        vec3 deltaDirection = normalize(dir) * delta;
        float deltaDirectionLength = length(deltaDirection);
        vec3 currentPosition = startPos;
    
        //The alpha value accumulated so far.
        float accumulatedAlpha = 0.0;
    
        //How long has the ray travelled so far.
        float accumulatedLength = 0.0;
    
        //vec4 dataSample;
        vec4 dataSample;
        float alphaSample;
        //Perform the ray marching iterations
        for(int i = 0; i < 81; i++){
            //Get the voxel intensity value from the 3D texture.    
            dataSample = getRGBAfromDataTex(dataTexture, currentPosition, dataShape, textureShape);
            //Store the alpha accumulated so far.
            accumulatedAlpha += dataSample.a;
        
            //Advance the ray.
            currentPosition += deltaDirection;
            accumulatedLength += deltaDirectionLength;
                      
            //If the length traversed is more than the ray length, or if the alpha accumulated reaches 1.0 then exit.
            if(accumulatedLength >= rayLength || accumulatedAlpha >= 1.0 ){
                break;
            }
        }
        return min(accumulatedAlpha, 1.0);
    }
	vec3 mapTo3D(vec3 pos, float sliceW, float sliceH, int nSlicesPerRow, int nSlices) {
		int sliceCol = int(floor(pos.x / sliceW));
		int sliceRow = int(floor(pos.y / sliceH));
        //Fix to put rows in the right order
        int sliceIndex = (((4 - sliceRow) * nSlicesPerRow) + (sliceCol));
		vec2 pxy = (pos.xy / vec2(sliceW, sliceH)) - vec2(sliceCol, sliceRow);
		//pos.z in {0,1,2}
		int z = ((int(pos.z) * nSlices) + sliceIndex);
		float pz = float(z)/90.0;
		return vec3(pxy, pz);
	}
	void main() {
	   // Look up a color from the texture.
	   vec4 inPixel = texture2D(dataTexture, v_texCoord);
       float diff = 0.01 * 4096.0;
	   vec3 rInCoord = vec3(v_texCoord*4096.0, 0.0);
	   vec3 gInCoord = vec3(v_texCoord*4096.0, 1.0);
	   vec3 bInCoord = vec3(v_texCoord*4096.0, 2.0);
	   vec3 rPoint = mapTo3D(rInCoord, sliceW, sliceH, nSlicesPerRow, nSlices);
	   vec3 bPoint = mapTo3D(bInCoord, sliceW, sliceH, nSlicesPerRow, nSlices);
	   vec3 gPoint = mapTo3D(gInCoord, sliceW, sliceH, nSlicesPerRow, nSlices);
	   float rOut = getPathRGBA(rPoint, lightPosition, 81.0, dataTexture);
	   float bOut = getPathRGBA(bPoint, lightPosition, 81.0, dataTexture);
	   float gOut = getPathRGBA(gPoint, lightPosition, 81.0, dataTexture);
	   vec3 absorbed = vec3(rOut, gOut, bOut);
	   vec3 light = vec3(1.0) - absorbed;
	   gl_FragColor = vec4(absorbed, 1.0);
       //gl_FragColor = vec4(v_texCoord, 0.0 ,1.0);
	} """

In [7]:
program = gloo.Program(vertex, fragment, count=4)
program['dataTexture']    = cloudTex
program['u_resolution'] = np.array([4096.0, 4096.0])

In [8]:
my_positions_array = np.array([ (0,0),   (0,4096),   (4096,0),   (4096,4096)   ])

In [9]:
program['a_position'] = gloo.VertexBuffer(my_positions_array.astype(np.float32))

In [11]:
@c.connect
def on_draw(event):
    gloo.clear((1,1,1,1))
    program.draw(gl.GL_TRIANGLE_STRIP)
    im = gloo.util._screenshot((0, 0, c.size[0], c.size[1]))
    imsave('/Users/rachel/Downloads/screenshot1000.png', im)
    c.close()

In [12]:
app.run();

INFO: Variable a_texCoord is not an active attribute
INFO:vispy:Variable a_texCoord is not an active attribute


(4096, 4096)
